<a href="https://colab.research.google.com/github/Niki-098/avkalan_llm/blob/master/Final_semantic_search_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Split_Pipeline work**

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import sqlite3
import numpy as np

# Try reading the CSV with a different encoding
questions_df = pd.read_csv('/content/WCG_study_id.csv', encoding='ISO-8859-1')

# Strip any whitespace from column names
questions_df.columns = questions_df.columns.str.strip()

# Rename columns for easier access
questions_df.rename(columns={'Question': 'question', 'Question Id': 'question_id'}, inplace=True)

# Preprocessing step - clean up the questions (strip unnecessary spaces)
questions_df['question'] = questions_df['question'].apply(lambda x: x.strip())

# Initialize the model for embedding generation (e.g., BERT)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each question
questions_df['embedding'] = questions_df['question'].apply(lambda x: model.encode(x))

# Connect to SQLite (or create a new one)
db_path = '/content/database.db'  # Update with your actual path
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create table for questions and their embeddings
cursor.execute('''
CREATE TABLE IF NOT EXISTS questions (
    question_id TEXT PRIMARY KEY,
    question TEXT,
    category TEXT,
    country TEXT,
    embedding BLOB
)
''')

# Insert data into the table
# # Insert data into the table, ignoring duplicates
# for _, row in questions_df.iterrows():
#     embedding_bytes = np.array(row['embedding']).tobytes()

#     cursor.execute('''
#     INSERT OR IGNORE INTO questions (question_id, question, category, country, embedding)
#     VALUES (?, ?, ?, ?, ?)
#     ''', (row['question_id'], row['question'], row['Category'], row['Country'], sqlite3.Binary(embedding_bytes)))

# conn.commit()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Insert data into the table, ignoring duplicates
for _, row in questions_df.iterrows():
    embedding_bytes = np.array(row['embedding']).tobytes()

    cursor.execute('''
    INSERT OR IGNORE INTO questions (question_id, question, category, country, embedding)
    VALUES (?, ?, ?, ?, ?)
    ''', (row['question_id'], row['question'], row['Category'], row['Country'], sqlite3.Binary(embedding_bytes)))

conn.commit()

In [ ]:
def filter_questions(category, country):
    query = f"""
    SELECT * FROM questions
    WHERE category = '{category}' AND country = '{country}'
    """
    return pd.read_sql(query, conn)

# Example of filtering by category 'Science' and country 'USA'
filtered_df = filter_questions('aGC: Landscape', 'none')
print(filtered_df)


   question_id                                           question  \
0            1  [Pre-Interview Worksheet] Please refer to Slid...   
1          1.a  [Pre-Interview Worksheet] What are your percep...   
2          1.b  [Pre-Interview Worksheet] [Payer] When you con...   
3        1.b.i  Moderator note: Hypotheses from market-specifi...   
4            2  [Pre-Interview Worksheet] Relative to other so...   
5          2.a  [Pre-Interview Worksheet] How does the clinica...   
6          2.b                       Probe rationale for ratings.   
7          2.c  What are the major drivers of clinical burden ...   
8        2.c.i  How do these drivers differ for 2L vs 3L+ pati...   
9          2.d  Do your perceptions of clinical burden differ ...   
10           3  [Pre-Interview Worksheet] Relative to other so...   
11         3.a  [Pre-Interview Worksheet] How does the economi...   
12         3.b                                    Probe rationale   
13         3.c  What are the major

**Retrieving most relevant Question ID**

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to perform semantic search
def semantic_search(query, filtered_df, model):
    # Encode the query to get its embedding
    query_embedding = model.encode(query)

    # Get embeddings for the filtered questions from the DataFrame
    embeddings = np.array([np.frombuffer(row.embedding, dtype=np.float32) for row in filtered_df.itertuples(index=False)])  # Use the column name

    # Calculate cosine similarity
    similarities = cosine_similarity([query_embedding], embeddings)

    # Get the index of the most similar question
    most_similar_idx = np.argmax(similarities)
    return filtered_df.iloc[most_similar_idx]['question_id']

# Example of semantic search
query = "major drivers of clinical burden for patients"
question_id = semantic_search(query, filtered_df, model)
print(f"Most relevant question ID: {question_id}")

Most relevant question ID: 2.c


**Retrieving relevant User ID**

In [ ]:
import pandas as pd

# Step 1: Load Data (Read the CSV file into a pandas DataFrame)
df = pd.read_csv('user_table.csv')  # Replace with your actual CSV file name

# Clean up column names (strip spaces)
df.columns = df.columns.str.strip()

# Step 2: Define a function to filter data based on username, stakeholder, and country_grouping
def run_query(query_conditions):
    # Filter the DataFrame based on the query conditions passed
    filtered_df = df.query(query_conditions)

    # If results are found, return the 'user_id'
    if not filtered_df.empty:
        return filtered_df['user_id']  # Adjust column name if needed
    else:
        return None

# Step 3: Get user input for username, stakeholder, and country_grouping
username_input = input("Enter the username (or leave blank for all): ").strip()
stakeholder_input = input("Enter the stakeholder: ").strip()
country_grouping_input = input("Enter the country grouping: ").strip()

# Step 4: Create the query conditions dynamically
if username_input:  # Include username in the query only if it is not empty
    query_conditions = (
        f"username == '{username_input}' and "
        f"stakeholder_type == '{stakeholder_input}' and "
        f"Country_grouping == '{country_grouping_input}'"
    )
else:  # Exclude username filtering if it is empty
    query_conditions = (
        f"stakeholder_type == '{stakeholder_input}' and "
        f"Country_grouping == '{country_grouping_input}'"
    )

# Run the query to get the relevant results from the DataFrame
results = run_query(query_conditions)

# Step 5: Store the results (user_ids) in a variable
user_ids = results.tolist() if results is not None else []

# Step 6: Display the results
if user_ids:
    print("Relevant User IDs:")
    print(user_ids)
else:
    print("No results found for the given inputs.")


Enter the username (or leave blank for all): 
Enter the stakeholder: GC_KOL
Enter the country grouping: EU
Relevant User IDs:
[1, 2, 3, 4]


**Retrieving Answers with the help of question_id and user_id**

In [ ]:
import pandas as pd

# Step 1: Load the answers table (replace with your actual file path)
answers_df = pd.read_csv('answer_table.csv')  # Replace with your answers CSV file path

# Clean up column names (strip spaces)
answers_df.columns = answers_df.columns.str.strip()

# Step 2: Assume these variables are already populated from previous steps
question_ids = ['2.c']  # Example question IDs
user_ids = [1]  # Example user IDs

# Step 3: Filter the answers based on the question_ids and user_ids
filtered_answers = answers_df[
    (answers_df['question_id'].isin(question_ids)) & (answers_df['user_id'].isin(user_ids))
]

# Save the filtered answers to a new CSV file
filtered_answers.to_csv('filtered_answers_table.csv', index=False)

# Step 4: Display the filtered answers
if not filtered_answers.empty:
    print("Answers:")
    for index, row in filtered_answers.iterrows():
        print(row['answer'])  # Assuming 'answer' is the column name for answers
else:
    print("No answers found for the given question IDs and user IDs.")


Answers:
[clarifying clinical burden] I think its logical the patient as an advanced disease the more the problem worsens and the quality of life gets bad â€“ so the more advanced, the higher the clinical burden. [anything symptom-wise you would pull out as relevant here?] in the first line treatment the objective is disease control and improving the survival. The more the disease progresses, the priority becomes QoL and symptoms and not really the survival. QoL is very important for patients in 3rd line treatment and survival more important in 1L treatment.


**All work in single Pipeline**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load the answers table (replace with your actual file path)
answers_df = pd.read_csv('answer_table.csv')  # Replace with your answers CSV file path
answers_df.columns = answers_df.columns.str.strip()  # Clean up column names

# Step 2: Load the questions table (replace with your actual file path)
questions_df = pd.read_csv('WCG_study_id.csv')  # Replace with your questions CSV file path
questions_df.columns = questions_df.columns.str.strip()  # Clean up column names

# Step 3: Function to filter questions based on category and country
def filter_questions(category, country):
    query = f"""
    SELECT * FROM questions
    WHERE category = '{category}' AND country = '{country}'
    """
    return pd.read_sql(query, conn)  # Assuming `conn` is your database connection

# Step 4: Semantic Search Function
def semantic_search(query, filtered_df, model):
    # Encode the query to get its embedding
    query_embedding = model.encode(query)

    # Ensure the embeddings column is processed correctly (based on the format in your data)
    embeddings = np.array([eval(row.embedding) if isinstance(row.embedding, str) else np.frombuffer(row.embedding, dtype=np.float32) for row in filtered_df.itertuples(index=False)])

    # Calculate cosine similarity
    similarities = cosine_similarity([query_embedding], embeddings)

    # Get the index of the most similar question
    most_similar_idx = np.argmax(similarities)
    return filtered_df.iloc[most_similar_idx]['question_id']

# Step 5: Query User Table based on username, stakeholder, and country grouping
def run_query(query_conditions, df):
    filtered_df = df.query(query_conditions)
    if not filtered_df.empty:
        return filtered_df['user_id']
    else:
        return None

# Step 6: Run the full pipeline
def run_pipeline(category_input, country_input, query_input, username_input, stakeholder_input, country_grouping_input, model):
    # Step 6.1: Filter the questions based on category and country
    filtered_questions = filter_questions(category_input, country_input)

    # Step 6.2: Perform semantic search to find the most similar question
    question_id = semantic_search(query_input, filtered_questions, model)

    # Step 6.3: Filter the user table based on user inputs
    df = pd.read_csv('user_table.csv')  # Replace with your actual CSV file name
    df.columns = df.columns.str.strip()

    # Step 6.4: Dynamically create the query condition for user filtering
    if username_input:  # Include username in the query only if it is not empty
        query_conditions = (
            f"username == '{username_input}' and "
            f"stakeholder_type == '{stakeholder_input}' and "
            f"Country_grouping == '{country_grouping_input}'"
        )
    else:  # Exclude username filtering if it is empty
        query_conditions = (
            f"stakeholder_type == '{stakeholder_input}' and "
            f"Country_grouping == '{country_grouping_input}'"
        )

    # Get the relevant user IDs based on query conditions
    results = run_query(query_conditions, df)

    # Step 6.5: Filter answers based on question_id and user_ids
    if results is not None:
        user_ids = results.tolist()
        filtered_answers = answers_df[
            (answers_df['question_id'] == question_id) & (answers_df['user_id'].isin(user_ids))
        ]

        # Save the filtered answers to a new CSV file
        filtered_answers.to_csv('filtered_answers_table.csv', index=False)

        # Step 6.6: Display the filtered answers (Only answer column)
        if not filtered_answers.empty:
            print("Answers:")
            for index, row in filtered_answers.iterrows():
                print(row['answer'])  # Assuming 'answer' is the column name for answers
        else:
            print("No answers found for the given question IDs and user IDs.")
    else:
        print("No user found for the given username, stakeholder, and country grouping.")

# Example of running the pipeline
category_input = 'aGC: Landscape'  # Example input
country_input = 'none'  # Example input
query_input = input("Ask question: ").strip() # Example input query
username_input = 'Aziz Zanaan'
stakeholder_input = 'GC_KOL'
country_grouping_input = 'EU'

# Assuming `model` is already loaded with your sentence-transformer or relevant model for encoding
run_pipeline(category_input, country_input, query_input, username_input, stakeholder_input, country_grouping_input, model)


Ask question: "What are the key factors contributing to the economic burden faced by patients in managing their condition?
Answers:
In FR we donâ€™t look at price of the drugs. We look at the drugs we have and treat with those


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests

# Step 1: Load the answers table (replace with your actual file path)
answers_df = pd.read_csv('answer_table.csv')  # Replace with your answers CSV file path
answers_df.columns = answers_df.columns.str.strip()  # Clean up column names

# Step 2: Load the questions table (replace with your actual file path)
questions_df = pd.read_csv('WCG_study_id.csv')  # Replace with your questions CSV file path
questions_df.columns = questions_df.columns.str.strip()  # Clean up column names

# Step 3: Function to filter questions based on category and country
def filter_questions(category, country):
    # Filter the questions DataFrame based on category and country
    filtered_questions = questions_df[
        (questions_df['category'] == category) &
        (questions_df['country'] == country)
    ]
    return filtered_questions

# Step 4: Semantic Search Function with Gemini LLM
def semantic_search(query, filtered_df, gemini_api_key, gemini_endpoint):
    """
    Perform semantic search using Gemini LLM for query embedding.
    """
    # Step 4.1: Get query embedding from Gemini
    try:
        response = requests.post(
            gemini_endpoint,
            headers={"Authorization": f"Bearer {gemini_api_key}", "Content-Type": "application/json"},
            json={"text": query}  # Assuming Gemini accepts a JSON with "text"
        )
        response.raise_for_status()
        query_embedding = np.array(response.json()["embedding"])
    except Exception as e:
        print(f"Error in Gemini API call: {e}")
        return None

    # Ensure the embeddings column is processed correctly
    embeddings = np.array([
        eval(row.embedding) if isinstance(row.embedding, str) else np.frombuffer(row.embedding, dtype=np.float32)
        for row in filtered_df.itertuples(index=False)
    ])

    # Calculate cosine similarity
    similarities = cosine_similarity([query_embedding], embeddings)

    # Get the index of the most similar question
    most_similar_idx = np.argmax(similarities)
    return filtered_df.iloc[most_similar_idx]['question_id']

# Step 5: Query User Table based on username, stakeholder, and country grouping
def run_query(query_conditions, df):
    filtered_df = df.query(query_conditions)
    if not filtered_df.empty:
        return filtered_df['user_id']
    else:
        return None

# Step 6: Run the full pipeline
def run_pipeline(category_input, country_input, query_input, username_input, stakeholder_input, country_grouping_input, gemini_api_key, gemini_endpoint):
    # Step 6.1: Filter the questions based on category and country
    filtered_questions = filter_questions(category_input, country_input)
    if filtered_questions.empty:
        print("No questions found for the given category and country.")
        return

    # Step 6.2: Perform semantic search using Gemini
    question_id = semantic_search(query_input, filtered_questions, gemini_api_key, gemini_endpoint)
    if question_id is None:
        print("Failed to find the most similar question. Please check the query or Gemini API settings.")
        return

    # Step 6.3: Load the user table
    df = pd.read_csv('user_table.csv')  # Replace with your actual CSV file name
    df.columns = df.columns.str.strip()

    # Step 6.4: Dynamically create the query condition for user filtering
    if username_input:  # Include username in the query only if it is not empty
        query_conditions = (
            f"username == '{username_input}' and "
            f"stakeholder_type == '{stakeholder_input}' and "
            f"Country_grouping == '{country_grouping_input}'"
        )
    else:  # Exclude username filtering if it is empty
        query_conditions = (
            f"stakeholder_type == '{stakeholder_input}' and "
            f"Country_grouping == '{country_grouping_input}'"
        )

    # Get the relevant user IDs based on query conditions
    results = run_query(query_conditions, df)

    # Step 6.5: Filter answers based on question_id and user_ids
    if results is not None:
        user_ids = results.tolist()
        filtered_answers = answers_df[
            (answers_df['question_id'] == question_id) & (answers_df['user_id'].isin(user_ids))
        ]

        # Save the filtered answers to a new CSV file
        filtered_answers.to_csv('filtered_answers_table.csv', index=False)

        # Step 6.6: Display the filtered answers (Only answer column)
        if not filtered_answers.empty:
            print("Answers:")
            for index, row in filtered_answers.iterrows():
                print(row['answer'])  # Assuming 'answer' is the column name for answers
        else:
            print("No answers found for the given question IDs and user IDs.")
    else:
        print("No user found for the given username, stakeholder, and country grouping.")

# Example of running the pipeline
if __name__ == "__main__":
    # Set your Gemini API details
    gemini_api_key = "your_gemini_api_key"  # Replace with your actual Gemini API key
    gemini_endpoint = "https://api.gemini.com/v1/embeddings"  # Replace with the actual Gemini embeddings endpoint

    # Example inputs
    category_input = 'aGC: Landscape'  # Example category
    country_input = 'none'  # Example country
    query_input = input("Ask question: ").strip()  # User input query
    username_input = 'Aziz Zanaan'
    stakeholder_input = 'GC_KOL'
    country_grouping_input = 'EU'

    run_pipeline(category_input, country_input, query_input, username_input, stakeholder_input, country_grouping_input, gemini_api_key, gemini_endpoint)
